In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Task A

Grayscale method was take from question 2's implementation. Here the ratios by which I multiply is based on this article: [NTSC Formula for Grayscale](https://support.ptc.com/help/mathcad/r10.0/en/index.html#page/PTC_Mathcad_Help/example_grayscale_and_color_in_images.html)

In [ ]:
def ICV_to_grayscale(image):
    w, h, c = image.shape
    grayimg = np.zeros((w,h))
    grayimg = (0.299*image[:,:,0] + 0.587*image[:,:,1] + 0.114*image[:,:,2])
    return grayimg

## Utility Functions for Question 4

In [ ]:
def ICV_divide_into_non_overlapping_windows(image, window_size):
    """
    Takes an image and divides it into windows of dimension window_size x window_size

    Parameters:
    image -> List[List[int]]
    window_size -> int

    Returns:
    List[ List[List[int]] ] (List of windows)
    """

    width, height = image.shape
    non_overlapping_windows = []
    for i in range(0, width, window_size):
        for j in range(0, height, window_size):
            window = image[i:i+window_size, j:j+window_size]
            non_overlapping_windows.append(window)
    return non_overlapping_windows

In [ ]:
def ICV_lbp_grayscale(image):
    """
    Takes an image and computes lbp codes for each pixel in the image. 

    Parameters:
    image -> List[List[int]]

    Returns:
    new_image -> List[List[int]] 
    """

    def ICV_get_lbp_code(window):
        """ 
        This function takes the 3x3 window matrix and then computes the LBP code for it. 
        Uses a string to build up the LBP code and then converts into a number. 
        
        
        Parameters: 
        window -> List[List[int]]

        Returns:
        int (base 10)
        """

        lbp_code = ""
        threshold = window[1][1] # center value acts as threshold
        for i, w in enumerate(window.flatten()): # flatten the array to speed up computation
            if i != 4: # in the flattened array, the center value in 3x3 matrix is at i = 4, so we skip it
                lbp_code += "1" if w > threshold else "0"
        return int(lbp_code, 2) # convert to base 10 number
        
    width, height = image.shape[:2]
    new_image = np.zeros((width, height))
    k_w, k_h = 3, 3

    # To apply LBP to each pixel I pad the image
    pad_width, pad_height = (k_w-1)//2, (k_h-1)//2 
    padded_image = np.zeros((width + 2 * pad_width, height + 2 * pad_height))
    padded_image[pad_width:pad_width + width, pad_height:pad_height + height] = image

    for i in range(width-k_w+1):
        for j in range(height-k_h+1):
            # Get LBP code for each window
            new_image[i+1, j+1] = ICV_get_lbp_code(padded_image[i:i+k_w, j:j+k_h])
    return new_image    

In [ ]:
def ICV_grayscale_histogram(image, bins=256):
    """
    Computes the raw and normalized histogram bins for an image

    Parameters: 
    image -> List[List[int]]
    bins -> int (optional)

    Returns: 
    List[]
    """
    height, width = image.shape
    histogram = np.zeros(bins)
    bin_width = 256 // bins
    for i in range(height):
        for j in range(width):
            bindex = min(int(image[i, j]) // bin_width, bins - 1)
            histogram[bindex] +=1
    histogram_n = [i/max(histogram) for i in histogram if max(histogram) != 0]
    return histogram, histogram_n

In [ ]:
def ICV_window_histograms(image, window_size):
    """
    Generates winodw histograms for an image given a specific window size

    Parameters:
    image -> List[List[int]]
    window_size -> int

    Returns: List[List[int]]
    """
    width, height = image.shape
    if width % window_size != 0 or height % window_size != 0:
        # window_size = np.gcd([width, height])
        print("Warning: Image dimensions are not divisible by window size. Extra pixels may be ignored.")
    hists = []
    for i in range(0, width, window_size):
        for j in range(0, height, window_size):
            window = image[i:i+window_size, j:j+window_size]
            hists.append(ICV_grayscale_histogram(window))
    return hists

In [ ]:
image = plt.imread("face-3.jpg")
image_g = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
plt.imshow(image_g, cmap="gray")

In [ ]:
window_size = 64
dinow = np.array(ICV_divide_into_non_overlapping_windows(image_g, window_size))
dinow_lbp = []
for i in dinow:
    dinow_lbp.append(ICV_lbp_grayscale(i))
dinow_lbp = np.array(dinow_lbp)
print(dinow_lbp.shape, dinow.shape)

In [ ]:
# Reshaping variables to aid with plotting the windows

dinow = dinow.reshape(4, 4, 64, 64)
dinow_lbp = dinow_lbp.reshape(4, 4, 64, 64)

fig, axs = plt.subplots(3, 2, figsize=(8, 8))
axs[0][0].imshow(dinow[0][0], cmap="gray")
axs[0][0].set_title("W1")
axs[0][1].imshow(dinow_lbp[0][0], cmap="gray")
axs[0][1].set_title("W2")
axs[1][0].imshow(dinow[2][2], cmap="gray")
axs[1][0].set_title("W3")
axs[1][1].imshow(dinow_lbp[2][2], cmap="gray")
axs[1][1].set_title("LBP1")
axs[2][0].imshow(dinow[3][2], cmap="gray")
axs[2][0].set_title("LBP2")
axs[2][1].imshow(dinow_lbp[3][2], cmap="gray")
axs[2][1].set_title("LBP3")
fig.suptitle("Random Windows from face-3.jpg and its lbp counterparts", fontsize=16)

for axs in axs.flat:
    axs.axis('off')

plt.savefig("Random Windows from face-3.jpg and its lbp counterparts.png")
plt.show()

fig1, axs = plt.subplots(4, 4, figsize=(5, 5), gridspec_kw={'hspace': 0.1, 'wspace': 0.1})
for i in range(4):
    for j in range(4):
        axs[i][j].imshow(dinow_lbp[i][j], cmap="gray")
        
for ax in axs.flat:
    ax.axis('off')
    
plt.show()

In [ ]:
dinow_lbp_hists = []
dinow_lbp_hists_n = []
for i in range(dinow_lbp.shape[0]):
    for j in range(dinow_lbp.shape[1]):
        hist, hist_n = ICV_grayscale_histogram(dinow_lbp[i][j])
        dinow_lbp_hists.append(hist)
        dinow_lbp_hists_n.append(hist_n)

In [ ]:
fig, axs = plt.subplots(3, figsize=(5, 5))
axs[0].plot(dinow_lbp_hists[0], color="b")
axs[1].plot(dinow_lbp_hists[10], color="b")
axs[2].plot(dinow_lbp_hists[14], color="b")
plt.tight_layout()
plt.savefig("Histograms of LBP Windows.png")
plt.show()

In [ ]:
# fig, axs = plt.subplots(len(dinow_lbp_hists_n), figsize=(5, 20))
# for i in range(len(dinow_lbp_hists_n)):
#     axs[i].plot(dinow_lbp_hists_n[i], color="b")
# plt.tight_layout()
# plt.show()

In [ ]:
for i in range(len(dinow_lbp_hists)):
    plt.plot(dinow_lbp_hists[i], color="b")
plt.show()

In [ ]:
for i in range(len(dinow_lbp_hists_n)):
    plt.plot(dinow_lbp_hists_n[i], color="b")
plt.show()

In [ ]:
# lbp_image = ICV_lbp_grayscale(image_g)
# print(lbp_image, lbp_image.shape)
# plt.imshow(lbp_image, cmap="gray")

In [ ]:
# window_size = 16
# hist = ICV_grayscale_histogram(lbp_image)
# wh = np.array(ICV_window_histograms(lbp_image, window_size))
# width, height = lbp_image.shape
# wh

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(hist, color="b")
# plt.xlabel("histogram bins")
# plt.ylabel("frequency")
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 5))
# plt.plot(wh, color="b")
# plt.xlabel("histogram bins")
# plt.ylabel("frequency")
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 5))
# for i in range(len(wh)):
#     plt.plot(wh[i], color="b")
#     plt.xlabel("histogram bins")
#     plt.ylabel("frequency")
# plt.show()

# Task B

### Generating Global Descriptors and Classifying Images using them

In [ ]:
def ICV_generate_global_descriptors(image_paths, window_size=64):
    """
    This function combines several utility functions to create global descriptor for all images which are given to it. 

    Parameters: 
    image_paths -> List[strings]
    window_size -> int

    Returns:
    List[List[int]], Tuple(int, int), Dictionary{string -> List[int]}  
    """

    global_descriptors = []
    path_to_desc = {}
    # Iterate through all images
    for i in image_paths:
        # Get the current Image
        image = plt.imread(i)
        # Convert it to grayscale
        image_g = ICV_to_grayscale(image) 
        # Divide it into non overlapping windows
        dinow = ICV_divide_into_non_overlapping_windows(image_g, window_size)
        dinow_lbp = []
        # Apply LBP to all of the windows for each image and store the results in a list
        for j in dinow:
            dinow_lbp.append(ICV_lbp_grayscale(j))
        dinow_lbp = np.array(dinow_lbp)
        dinow_lbp_hists_n = []
        # Get normalized histograms for each LBP Window and store it 
        for k in range(dinow_lbp.shape[0]):
            _, hist_n = ICV_grayscale_histogram(dinow_lbp[k])
            dinow_lbp_hists_n.append(hist_n)
        # Append the histograms togther and then flatten the list containing them to have one sparse list 
        # which represent the global descriptor for the image 
        global_descriptors.append(np.array(dinow_lbp_hists_n).flatten())
        # Add each image's associated descriptor in a dictionary to help with heatmap creaton
        path_to_desc[i] = path_to_desc.get(i, 0) + np.array(dinow_lbp_hists_n).flatten()
    return np.array(global_descriptors),  path_to_desc

image_paths = ["face-1.jpg", "face-2.jpg", "face-3.jpg", "car-1.jpg", "car-2.jpg", "car-3.jpg"]
gd1 = ICV_generate_global_descriptors(image_paths, window_size = 64)
gd2 = ICV_generate_global_descriptors(image_paths, window_size = 16)
gd3 = ICV_generate_global_descriptors(image_paths, window_size = 128)
# gd1[2], gd2[2], gd3[2]
gd1[0]

In [ ]:
def ICV_all_global_descriptor_similarities(descriptors):
    """
    Takes a list of all global descriptors and finds the intersection for all combinations of them taken 2 at a time

    Parameters: 
    descriptors -> List[List[int]]

    Returns:
    intersections -> Dictionary{Tuple(int, int) -> List[int]}
    """
    def ICV_descriptor_intersection(desc1, desc2):
        return np.sum(np.minimum(desc1, desc2))

    intersections = {}
    for i in range(len(descriptors)):
        for j in range(len(descriptors)):
            if (i,j) not in intersections and (j,i) not in intersections:
                if i == j:
                    intersections[(i, j)] = 1
                    continue
                d1 = descriptors[i]
                d2 = descriptors[j]
                i1 = ICV_descriptor_intersection(d1, d2)
                intersections[(i,j)] = i1
    return intersections

In [ ]:
gdi1 = ICV_all_global_descriptor_similarities(gd1[0])
gdi2 = ICV_all_global_descriptor_similarities(gd2[0])
gdi3 = ICV_all_global_descriptor_similarities(gd3[0])

gdi1, gdi2, gdi3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Here I generate a heatmap of the descriptor intersections between images to classify them for window size = 64

intersection_data = gdi1

num_images = len(image_paths) 

intersection_matrix = np.zeros((num_images, num_images))

for (i, j), value in intersection_data.items():
    intersection_matrix[i, j] = value
    intersection_matrix[j, i] = value 


plt.figure(figsize=(8, 6))
sns.heatmap(intersection_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
            xticklabels=[f'Image {i}' for i in range(num_images)], 
            yticklabels=[f'Image {i}' for i in range(num_images)])

plt.title('Global Descriptor Intersection Heatmap')
plt.xlabel('Images')
plt.ylabel('Images')
plt.tight_layout()
plt.savefig("Global_Descriptor_Intersection_for_Different_Images_in_DataSet_A.png")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Here I generate a heatmap of the descriptor intersections between images for all window sizes at once

intersection_data = [gdi1, gdi2, gdi3] # Global Descriptors as Dictionaries for window sizes 64, 16 and 128
num_images = len(image_paths)
intersection_matrix = np.zeros((num_images, num_images))

for k, i_d in enumerate(intersection_data):
    for (i, j), value in i_d.items():
        intersection_matrix[i, j] = value
        intersection_matrix[j, i] = value

    plt.figure(figsize=(8, 6))
    sns.heatmap(intersection_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
                xticklabels=[f'Image {i}' for i in range(num_images)], 
                yticklabels=[f'Image {i}' for i in range(num_images)])
    
    plt.title('Global Descriptor Intersection Heatmap')
    plt.xlabel('Images')
    plt.ylabel('Images')
    plt.tight_layout()
    plt.savefig(f"Global_Descriptor_Intersection_for_Different_Images_in_DataSet_A_Window_{k}.png")
    plt.show()

The idea with these heatmaps was to compare all images with each other at once. Here we can treat one image as reference and the other as the one which we want to classify. Based on that the ones which have highest similarity with the same class of images as themselves have correct classifications while ones with high intersections with the opposite classes are Misclassifications. For different window sizes the overall threshold for the score at which a window is a Face or not changes (the smaller the window the higher the score required).  Looking at the Heatmaps above it's evident that:
- Descriptors with smaller window sizes capture more information about subtle texture changes in the image which larger window sizes don't allow for. 
- This type of descriptor seems to perform fairly well for face images (based on higher similarity scores between face images against face images compared to car images against each other).
- As can be seen in the heatmap the model is most certain about the similarity of images 1 and 2 and 0 and 2. Thus Image 2 (face-3.jpg) can serve as a good baseline for face images. Similarly for Car images this is true with Image 4 in the heatmap or (car-2.jpg) a similar build type to (car-3.jpg).

In [ ]:
from matplotlib.pyplot import figure
face3 = plt.imread("face-3.jpg")
car1 = plt.imread("car-1.jpg")

fig, axs = plt.subplots(1, 2, figsize=(13, 5))
axs[0].imshow(face3)
axs[1].imshow(car1)
fig.suptitle("Example Images", fontsize=20)
plt.show()

for (i,j) in [(gd1, 64), (gd2, 16), (gd3, 128)]:
    i_2 = np.array(i[0][2])
    i_3 = np.array(i[0][3])
    fig, axs = plt.subplots(1, 2, figsize=(30, 10))
    axs[0].plot(i_2)
    axs[1].plot(i_3)
    fig.suptitle(f"Associated Global Descriptor Histograms (window size = {j})", fontsize=20)
    plt.show()